# 少量示例的提示词模板的使用

FewShotPromptTemplate: 与PromptTemplate一起使用

FewShotChatMessagePromptTemplate：与ChatPromptTemplate一起使用

Example selectors(示例选择器):

## 1、FewShotPromptTemplate的使用


举例1：未提供示例的情况

In [1]:

import os
import dotenv
from langchain_core.prompts import FewShotPromptTemplate
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"),
                        temperature=0.7)

res = chat_model.invoke("2 🦜 9是多少?")
print(res.content)

您的表达可能需要一些澄清。如果您是想要进行数学运算，那么2加上9等于11。如果您是指其他含义，请提供更多信息，我将很乐意帮助您。


举例2：使用FewShotPromptTemplate



In [24]:
from langchain_core.prompts import PromptTemplate

# 创建PromptTemplate的实例
example_prompt = PromptTemplate.from_template(
    template="input:{input}\noutput:{output}",
)

# 提供一些示例
examples = [
    {"input": "长沙天气怎么样？", "output": "长沙市"},
    {"input": "南京下雨吗？", "output": "南京市"},
    {"input": "武汉热吗？", "output": "武汉市"}
]

# 创建FewShotPromptTemplate的实例
few_shot_template = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples = examples,
    suffix="input:{input}\noutput:", # 声明在示例后面的提示词模板
    input_variables=["input"],
)

few_shot_template.invoke({"input":"杭州会下雨吗？"})

StringPromptValue(text='input:长沙天气怎么样？\noutput:长沙市\n\ninput:南京下雨吗？\noutput:南京市\n\ninput:武汉热吗？\noutput:武汉市\n\ninput:杭州会下雨吗？\noutput:')

调用大模型以后：

In [27]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"), temperature=0.4)

chat_model.invoke(few_shot_template.invoke({"input":"杭州会下雨吗？"}))


AIMessage(content='杭州市', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 68, 'total_tokens': 69, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen2.5-7B-Instruct', 'system_fingerprint': '', 'id': '019bf409bae898789574b7e88db76545', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bf409-c784-7fd1-90a7-9919f0684162-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 68, 'output_tokens': 1, 'total_tokens': 69, 'input_token_details': {}, 'output_token_details': {}})

举例3：


In [3]:
from langchain_core.prompts import PromptTemplate

# 提示模板，配置一个提示模板，将一个示例格式化为字符串
prompt_template = "你是一个数学专家,算式： {input} 值： {output} 使用： {description} "

# 这是一个提示模板，用于设置每个示例的格式
prompt_sample = PromptTemplate.from_template(prompt_template)

# 2、提供示例
examples = [
    {"input": "2+2", "output": "4", "description": "加法运算"},
    {"input": "5-2", "output": "3", "description": "减法运算"},
]

# 3、创建一个FewShotPromptTemplate对象
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_sample,
    suffix="你是一个数学专家,算式: {input}  值: {output}",
    input_variables=["input", "output"]
)
# print(prompt.invoke({"input":"2*5", "output":"10"}))

# 4、初始化大模型，然后调用
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

result = chat_model.invoke(prompt.invoke({"input":"2*5", "output":""}))
print(result.content)  # 使用: 乘法运算

你是一个数学专家,算式: 2*5 值: 10 使用： 乘法运算 

解析：这个算式是2乘以5，根据乘法运算的规则，2乘以5的结果是10。在这个例子中，2是乘数，5也是乘数，它们相乘的结果就是10。


# 2、FewShotChatMessagePromptTemplate的使用

举例1：实例化

In [1]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

# 1.示例消息格式
examples = [
    {"input": "1+1等于几？", "output": "1+1等于2"},
    {"input": "法国的首都是？", "output": "巴黎"}
]

# 2.定义示例的消息格式提示词模版
msg_example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

# 3.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=msg_example_prompt,
    examples=examples
)

# 4.输出格式化后的消息
print(few_shot_prompt.format())

Human: 1+1等于几？
AI: 1+1等于2
Human: 法国的首都是？
AI: 巴黎


举例2：

In [2]:
# 1.导入相关包
from langchain_core.prompts import (FewShotChatMessagePromptTemplate, ChatPromptTemplate)

# 2.定义示例组
examples = [
    {"input": "2*2", "output": "4"},
    {"input": "2*3", "output": "6"},
]

# 3.定义示例的消息格式提示词模版
example_prompt = ChatPromptTemplate.from_messages([
    ('human', '{input} 是多少?'),
    ('ai', '{output}')
])

# 4.定义FewShotChatMessagePromptTemplate对象
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,  # 示例组
    example_prompt=example_prompt,  # 示例提示词词模版
)

# 5.输出完整提示词的消息模版
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个数学奇才'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

# 6.提供大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

chat_model = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"),
                        temperature=0.5)

chat_model.invoke(final_prompt.invoke(input="2*4")).content

'8'

## 3、Example selectors(示例选择器)

举例1：

In [1]:
# 1.导入相关包
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
import os
import dotenv
from langchain_openai import OpenAIEmbeddings

dotenv.load_dotenv()

# 2.定义嵌入模型
os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

embeddings_model = OpenAIEmbeddings(
    model=os.getenv("LLM_EMBEDDING_MODEL_ID")
)

# 3.定义示例组
examples = [
    {
        "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵?",
        "answer": """
        接下来还需要问什么问题吗？
        追问：穆罕默德·阿里去世时多大年纪？
        中间答案：穆罕默德·阿里去世时享年74岁。
        """,
    },
    {
        "question": "craigslist的创始人是什么时候出生的？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：谁是craigslist的创始人？
        中级答案：Craigslist是由克雷格·纽马克创立的。
        """,
    },
    {
        "question": "谁是乔治·华盛顿的外祖父？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：谁是乔治·华盛顿的母亲？
        中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。
        """,
    },
    {
        "question": "《大白鲨》和《皇家赌场》的导演都来自同一个国家吗？",
        "answer": """
        接下来还需要问什么问题吗？
        追问：《大白鲨》的导演是谁？
        中级答案：《大白鲨》的导演是史蒂文·斯皮尔伯格。
        """,
    },
]

# 4.定义示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 这是可供选择的示例列表
    examples,
    # 这是用于生成嵌入的嵌入类，用于衡量语义相似性
    embeddings_model,
    # 这是用于存储嵌入并进行相似性搜索的 VectorStore 类
    Chroma,
    # 这是要生成的示例数量
    k=1,
)

# 选择与输入最相似的示例
question = "玛丽·鲍尔·华盛顿的父亲是谁?"
selected_examples = example_selector.select_examples({"question": question})
print(f"与输入最相似的示例：{selected_examples}")

# for example in selected_examples:
#     print("\n")
#     for k, v in example.items():
#         print(f"{k}: {v}")

与输入最相似的示例：[{'answer': '\n        接下来还需要问什么问题吗？\n        追问：谁是乔治·华盛顿的母亲？\n        中间答案：乔治·华盛顿的母亲是玛丽·鲍尔·华盛顿。\n        ', 'question': '谁是乔治·华盛顿的外祖父？'}]


举例2：

In [16]:
# 1.导入相关包
import os
import dotenv
# from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

# 2.定义示例提示词模版
example_prompt = PromptTemplate.from_template(
    template="Input: {input}\nOutput: {output}",
)

# 3.创建一个示例提示词模版
examples = [
    {"input": "高兴", "output": "悲伤"},
    {"input": "高", "output": "矮"},
    {"input": "长", "output": "短"},
    {"input": "精力充沛", "output": "无精打采"},
    {"input": "阳光", "output": "阴暗"},
    {"input": "粗糙", "output": "光滑"},
    {"input": "干燥", "output": "潮湿"},
    {"input": "富裕", "output": "贫穷"},
]

# 4.定义嵌入模型
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")
embeddings = OpenAIEmbeddings(
    model=os.getenv("LLM_EMBEDDING_MODEL_ID")
)

# 5.创建语义相似性示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    Chroma,
    k=1,
    input_keys=["input"],  # 指定用于生成嵌入的字段
)

# 或者
#example_selector = SemanticSimilarityExampleSelector(
#    examples,
#    embeddings,
#    FAISS,
#    k=2
#)

# 6.定义小样本提示词模版
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个词组的反义词",
    suffix="Input: {input}\nOutput:",
    input_variables=["input"],
)

response = similar_prompt.invoke({"input":"忧郁"})
print(response.text)

给出每个词组的反义词

Input: 干燥
Output: 潮湿

Input: 忧郁
Output:
